In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json
import pprint

import pathlib
import logging

In [2]:
# Use start time from info file (instead of recording.meta_info.start_time_synced_ns)
# to have a more precise value and avoid having a negative first timestamp when rewriting
# start_time_synced_ns = int(info_json["start_time"])
# start_time_synced_ns = 1706217550686000000

def conversion(timestamps):
    # Subtract start_time from all times in the recording, so timestamps
    # start at 0. This is to increase precision when converting
    # timestamps to float32, e.g. for OpenGL!
    SECONDS_PER_NANOSECOND = 1e-9
    # return (timestamps - start_time_synced_ns) * SECONDS_PER_NANOSECOND
    return timestamps * SECONDS_PER_NANOSECOND

In [3]:
def _rewrite_times(path, dtype="<u8"):
    """Load raw times (assuming dtype), apply conversion and save as _timestamps.npy."""
    
    timestamps = np.fromfile(str(path), dtype=dtype)
    new_name = f"{path.stem}_timestamps_unix.npy"
    timestamp_loc = path.parent / new_name
    np.save(str(timestamp_loc), timestamps)

    timestamps = conversion(timestamps)

    new_name = f"{path.stem}_timestamps.npy"
    # logger.info(f"Creating {new_name}")
    timestamp_loc = path.parent / new_name
    np.save(str(timestamp_loc), timestamps)

In [4]:
def _load_timestamps_data(path):
    timestamps = np.load(str(path))
    return timestamps.astype(np.float64)

In [5]:
def _neon_raw_time_load(path):
    return np.fromfile(str(path), dtype="<u8")

In [6]:
def _load_raw_data(path):
    raw_data = np.fromfile(str(path), "<f4")
    raw_data_dtype = raw_data.dtype
    raw_data.shape = (-1, 2)
    return np.asarray(raw_data, dtype=raw_data_dtype)

In [7]:
def _load_worn_data(path: pathlib.Path):
    if not (path and path.exists()):
        return None

    confidences = np.fromfile(str(path), "<u1") / 255.0
    return np.clip(confidences, 0.0, 1.0)

In [8]:
# obtained here:
# https://github.com/pupil-labs/neon-player/blob/master/pupil_src/shared_modules/imu_timeline/imu_timeline.py#L72
from scipy.spatial.transform import Rotation

import imu_pb2

logger = logging.getLogger(__name__)

def parse_neon_imu_raw_packets(buffer):
    index = 0
    packet_sizes = []
    while True:
        nums = np.frombuffer(buffer[index : index + 2], np.uint16)
        if not nums:
            break
        index += 2
        packet_size = nums[0]
        packet_sizes.append(packet_size)
        packet_bytes = buffer[index : index + packet_size]
        index += packet_size
        packet = imu_pb2.ImuPacket()
        packet.ParseFromString(packet_bytes)
        yield packet


class IMURecording:
    DTYPE_RAW = np.dtype(
        [
            ("gyro_x", "<f4"),
            ("gyro_y", "<f4"),
            ("gyro_z", "<f4"),
            ("accel_x", "<f4"),
            ("accel_y", "<f4"),
            ("accel_z", "<f4"),
            ("pitch", "<f4"),
            ("yaw", "<f4"),
            ("roll", "<f4"),
            ("quaternion_w", "<f4"),
            ("quaternion_x", "<f4"),
            ("quaternion_y", "<f4"),
            ("quaternion_z", "<f4"),
            ("tsNs", "uint64"),
            ("ts", "<f8"),
            ("ts_rel", "<f8")
        ]
    )

    def __init__(self, path_to_imu_raw: pathlib.Path, start_ts):
        stem = path_to_imu_raw.stem
        self.path_raw = path_to_imu_raw
        self.path_ts = path_to_imu_raw.with_name(stem + "_timestamps.npy")
        self.load(start_ts)

    def load(self, start_ts):
        if not self.path_raw.exists() and self.path_ts.exists():
            self.ts = np.empty(0, dtype=np.float64)
            self.raw = []
            return

        self.ts = np.load(str(self.path_ts))
        with self.path_raw.open('rb') as raw_file:
            raw_data = raw_file.read()
            imu_packets = parse_neon_imu_raw_packets(raw_data)
            imu_data = []
            for packet in imu_packets:
                rotation = Rotation.from_quat([packet.rotVecData.x, packet.rotVecData.y, packet.rotVecData.z, packet.rotVecData.w])
                euler = rotation.as_euler(seq='XZY', degrees=True)

                ts = conversion(float(packet.tsNs))
                ts_rel = ts - start_ts

                imu_data.append((
                    packet.gyroData.x, packet.gyroData.y, packet.gyroData.z,
                    packet.accelData.x, packet.accelData.y, packet.accelData.z,
                    *euler,
                    packet.rotVecData.w, packet.rotVecData.x, packet.rotVecData.y, packet.rotVecData.z,
                    packet.tsNs,
                    ts,
                    ts_rel
                ))

            self.raw = np.array(imu_data, dtype=IMURecording.DTYPE_RAW).view(
                np.recarray
            )

        num_ts_during_init = self.ts.size - len(self.raw)
        if num_ts_during_init > 0:
            self.ts = self.ts[num_ts_during_init:]

In [11]:
start_ts = conversion(1706217550686000000)

with pathlib.Path('../tests/test_data/2024-01-25_22-19-10_test-f96b6e36/extimu ps1.raw').open('rb') as raw_file:
            raw_data = raw_file.read()
            imu_packets = parse_neon_imu_raw_packets(raw_data)
            imu_data = []
            for packet in imu_packets:
                rotation = Rotation.from_quat([packet.rotVecData.x, packet.rotVecData.y, packet.rotVecData.z, packet.rotVecData.w])
                euler = rotation.as_euler(seq='XZY', degrees=True)
                
                ts = conversion(packet.tsNs)
                ts_rel = ts - start_ts

                imu_data.append((
                    packet.gyroData.x, packet.gyroData.y, packet.gyroData.z,
                    packet.accelData.x, packet.accelData.y, packet.accelData.z,
                    *euler,
                    packet.rotVecData.w, packet.rotVecData.x, packet.rotVecData.y, packet.rotVecData.z,
                    packet.tsNs,
                    ts,
                    ts_rel
                ))

            raw = np.array(imu_data, dtype=IMURecording.DTYPE_RAW).view(
                np.recarray
            )

/var/folders/by/f4g6q8690z50zbfvy6kq71cc0000gn/T/ipykernel_80601/647576048.py:14: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if not nums:


In [12]:
raw

rec.array([( 0.10108948, -0.53596497, 0.70762634, 0.04443359, -0.6230469 , 0.7832031 , -118.563385, -44.58207 , 124.69439 , 0.50820255, -0.1975484 , 0.2673142 , -0.7945111 , 1706217553148771064, 1.70621755e+09,  2.46277094),
           ( 0.2822876 , -0.47683716, 0.5893707 , 0.04882812, -0.6269531 , 0.78271484, -118.54282 , -44.59728 , 124.66329 , 0.5083921 , -0.19759709, 0.26724702, -0.7944003 , 1706217553159015064, 1.70621755e+09,  2.47301483),
           ( 0.2822876 , -0.11062622, 0.5245209 , 0.04150391, -0.62841797, 0.77539057, -118.5168  , -44.61387 , 124.62134 , 0.5086237 , -0.19768989, 0.26716286, -0.7942572 , 1706217553169259064, 1.70621755e+09,  2.48325872),
           ...,
           (-0.78201294, -2.6168823 , 2.653122  , 0.11816406, -0.8154297 , 0.5932617 , -107.888054, -33.64299 , 110.22006 , 0.51416355, -0.30293173, 0.32829908, -0.7321802 , 1706217591021952064, 1.70621759e+09, 40.33595181),
           (-1.2111664 , -2.2525787 , 2.7179718 , 0.12597656, -0.80615234, 0.6108398

In [13]:
_rewrite_times(pathlib.Path('../tests/test_data/2024-01-25_22-19-10_test-f96b6e36/extimu ps1.time'))
imu_rec = IMURecording(pathlib.Path('../tests/test_data/2024-01-25_22-19-10_test-f96b6e36/extimu ps1.raw'), conversion(1706217550686000000))

/var/folders/by/f4g6q8690z50zbfvy6kq71cc0000gn/T/ipykernel_80601/647576048.py:14: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if not nums:


In [14]:
[imu_rec.raw.ts[0], imu_rec.raw.ts[1]]

[1706217553.1487713, 1706217553.1590152]

In [15]:
gaze_ps1_raw = _load_raw_data('../tests/test_data/2024-01-25_22-19-10_test-f96b6e36/gaze ps1.raw')
gaze_ps1_time = _neon_raw_time_load('../tests/test_data/2024-01-25_22-19-10_test-f96b6e36/gaze ps1.time')

gaze_200hz_raw = _load_raw_data('../tests/test_data/2024-01-25_22-19-10_test-f96b6e36/gaze_200hz.raw')
gaze_200hz_time = _neon_raw_time_load('../tests/test_data/2024-01-25_22-19-10_test-f96b6e36/gaze_200hz.time')

gaze_right_ps1_raw = _load_raw_data('../tests/test_data/2024-01-25_22-19-10_test-f96b6e36/gaze_right ps1.raw')
gaze_right_ps1_time = _neon_raw_time_load('../tests/test_data/2024-01-25_22-19-10_test-f96b6e36/gaze_right ps1.time')

# '../tests/test_data/2024-01-25_22-19-10_test-f96b6e36/Neon Scene Camera v1 ps1.time'
# '../tests/test_data/2024-01-25_22-19-10_test-f96b6e36/Neon Scene Camera v1 ps1.time_aux'

# '../tests/test_data/2024-01-25_22-19-10_test-f96b6e36/Neon Sensor Module Camera v1 ps1.time'
# '../tests/test_data/2024-01-25_22-19-10_test-f96b6e36/Neon Sensor Module Camera v1 ps1.time_aux'

# imu_ps1_raw = _load_raw_data('../tests/test_data/2024-01-25_22-19-10_test-f96b6e36/extimu ps1.raw')
# imu_ps1_time = _neon_raw_time_load('../tests/test_data/2024-01-25_22-19-10_test-f96b6e36/extimu ps1.time')

worn_ps1_raw = _load_worn_data(pathlib.Path('../tests/test_data/2024-01-25_22-19-10_test-f96b6e36/worn ps1.raw'))
# worn_200hz_raw = _neon_raw_time_load('../tests/test_data/2024-01-25_22-19-10_test-f96b6e36/worn_200hz.raw')

In [16]:
worn_ps1_raw

array([1., 1., 1., ..., 1., 1., 1.])

In [17]:
class Stream():
    def __init__(self, name):
        self.name = name
        self.data = []
        self.ts = []
        self.ts_rel = []

    def load(self, data):
        self.data = data[:]
        self.ts = data[:].ts
        self.ts_rel = data[:].ts_rel

    def sample(self, tstamps):
        if len(tstamps) == 1:
            if tstamps < self.ts[0]:
                self.closest_idxs = [None]
            
            if tstamps > self.ts[-1]:
                self.closest_idxs = [None]
            
            self.closest_idxs = np.searchsorted(self.ts, tstamps)

        sorted_tses = np.sort(tstamps)

        self.closest_idxs = []
        for tc in range(1, len(sorted_tses)):
            prev_ts = sorted_tses[tc - 1]
            curr_ts = sorted_tses[tc]

            bounded_tstamps = self.ts[(self.ts >= prev_ts) & (self.ts <= curr_ts)]

            if len(bounded_tstamps) == 0:
                self.closest_idxs.append(None)
                continue

            # just always take the one that is closest to the current timestamp
            closest_ts = bounded_tstamps[-1]

            # this has the feeling of suboptimal
            self.closest_idxs.append(np.where(self.ts == closest_ts)[0][0])

    def __iter__(self):
        self.idx = 0
        return self
    
    def __next__(self):
        if self.idx < len(self.closest_idxs):
            idx = self.closest_idxs[self.idx]

            if idx is None:
                self.idx += 1
                return None
            else:
                self.idx += 1
                return self.data[idx:idx+1]
        else:
            raise StopIteration

In [18]:
# def sample(self, tstamps):
#     if len(tstamps) == 1:
#         if tstamps < self.ts[0]:
#             self.closest_idxs = [None]
        
#         if tstamps > self.ts[-1]:
#             self.closest_idxs = [None]
        
#         self.closest_idxs = np.searchsorted(self.ts, tstamps)

#     sorted_tses = np.sort(tstamps)

#     self.closest_idxs = []
#     for tc in range(1, len(sorted_tses)):
#         prev_ts = sorted_tses[tc - 1]
#         curr_ts = sorted_tses[tc]

#         bounded_tstamps = self.ts[(self.ts >= prev_ts) & (self.ts <= curr_ts)]

#         if len(bounded_tstamps) == 0:
#             self.closest_idxs.append(None)
#             continue

#         # just always take the one that is closest to the current timestamp
#         closest_ts = bounded_tstamps[-1]

#         # this has the feeling of suboptimal
#         self.closest_idxs.append(np.where(self.ts == closest_ts)[0][0])

# np.recarray.sample = sample

In [19]:
def convert_gaze_to_recarray(gaze_data, ts, ts_rel):
    out = np.recarray(gaze_data.shape[0], dtype=[('x', '<f4'), ('y', '<f4'), ('ts', '<f8'), ('ts_rel', '<f8')])
    out.x = gaze_data[:, 0]
    out.y = gaze_data[:, 1]
    out.ts = ts.astype(np.float64)
    out.ts_rel = ts_rel.astype(np.float64)
    
    return out

In [20]:
ts

1706217591.04255

In [22]:
class NeonRecording:
    def __init__(self):
        self.info = []
        self._calib = []

        self.start_ts = 0

        self.scene_camera = []
        self.eye1_camera = []
        self.eye2_camera = []
        self.streams = {
            'gaze': Stream('gaze'),
            'imu': Stream('imu'),
            'scene': Stream('scene')
        }

        self._gaze_ps1_raw_time = []
        self._gaze_200hz_raw_time = []
        self._gaze_right_ps1_raw_time = []
        self._gaze_ps1_ts = []
        self._gaze_ps1_raw = []
        self._gaze_right_ps1_ts = []
        self._gaze_right_ps1_raw = []
        self._worn_ps1_raw = []

    # TODO: save for the end
    def check(self):
        pass

    @property
    def gaze(self):
        return self.streams['gaze']
    
    @property
    def imu(self):
        return self.streams['imu']
    
    @property
    def scene(self):
        return self.streams['scene']

    @staticmethod
    def load(rec_dir):
        rec = NeonRecording()

        with open(rec_dir + '/info.json') as f:
            rec.info = json.load(f)

        rec._start_ts_ns = rec.info['start_time']
        rec.start_ts = conversion(rec.info['start_time'])

        with open(rec_dir + 'calibration.bin', 'rb') as f:
            raw_data = f.read()

        # obtained here: 
        # https://github.com/pupil-labs/realtime-python-api/blob/main/src/pupil_labs/realtime_api/device.py#L178
        rec._calib = np.frombuffer(
            raw_data,
            np.dtype(
                [
                    ("version", "u1"),
                    ("serial", "6a"),
                    ("scene_camera_matrix", "(3,3)d"),
                    ("scene_distortion_coefficients", "8d"),
                    ("scene_extrinsics_affine_matrix", "(4,4)d"),
                    ("right_camera_matrix", "(3,3)d"),
                    ("right_distortion_coefficients", "8d"),
                    ("right_extrinsics_affine_matrix", "(4,4)d"),
                    ("left_camera_matrix", "(3,3)d"),
                    ("left_distortion_coefficients", "8d"),
                    ("left_extrinsics_affine_matrix", "(4,4)d"),
                    ("crc", "u4"),
                ]
            ),
        )

        # rec.version = rec._calib['version']
        rec.serial = rec._calib['serial']
        rec.scene_camera = {
            'matrix': rec._calib['scene_camera_matrix'],
            'distortion': rec._calib['scene_distortion_coefficients'],
            'extrinsics': rec._calib['scene_extrinsics_affine_matrix']
        }
        rec.eye1_camera = {
            'matrix': rec._calib['right_camera_matrix'],
            'distortion': rec._calib['right_distortion_coefficients'],
            'extrinsics': rec._calib['right_extrinsics_affine_matrix']
        }
        rec.eye2_camera = {
            'matrix': rec._calib['left_camera_matrix'],
            'distortion': rec._calib['left_distortion_coefficients'],
            'extrinsics': rec._calib['left_extrinsics_affine_matrix']
        }

        rec._gaze_ps1_raw_time = _neon_raw_time_load(rec_dir + '/gaze ps1.time')
        rec._gaze_200hz_raw_time = _neon_raw_time_load(rec_dir + '/gaze_200hz.time')
        rec._gaze_right_ps1_raw_time = _neon_raw_time_load(rec_dir + '/gaze_right ps1.time')

        _rewrite_times(pathlib.Path(rec_dir + '/gaze ps1.time'))
        rec._gaze_ps1_ts = _load_timestamps_data(pathlib.Path(rec_dir + '/gaze ps1_timestamps.npy'))
        rec._gaze_ps1_raw = _load_raw_data(rec_dir + '/gaze ps1.raw')

        _rewrite_times(pathlib.Path(rec_dir + '/gaze_200hz.time'))
        gaze_200hz_ts = _load_timestamps_data(pathlib.Path(rec_dir + '/gaze_200hz_timestamps.npy'))
        gaze_200hz_raw = _load_raw_data(rec_dir + '/gaze_200hz.raw')

        gaze_200hz_ts_rel = gaze_200hz_ts - rec.start_ts
        rec.streams['gaze'].load(convert_gaze_to_recarray(gaze_200hz_raw, gaze_200hz_ts, gaze_200hz_ts_rel))

        _rewrite_times(pathlib.Path(rec_dir + '/gaze_right ps1.time'))
        rec._gaze_right_ps1_ts = _load_timestamps_data(pathlib.Path(rec_dir + '/gaze_right ps1_timestamps.npy'))
        rec._gaze_right_ps1_raw = _load_raw_data(rec_dir + '/gaze_right ps1.raw')

        _rewrite_times(pathlib.Path(rec_dir + '/extimu ps1.time'))
        imu_rec = IMURecording(pathlib.Path(rec_dir + '/extimu ps1.raw'), rec.start_ts)
        rec.streams['imu'].load(imu_rec.raw)

        rec._worn_ps1_raw = _load_worn_data(pathlib.Path(rec_dir + '/worn ps1.raw'))
        # worn_ps1_raw = _load_worn_data(pathlib.Path(rec_dir + '/worn_200hz.raw'))

        return rec

In [24]:
rec = NeonRecording.load('../tests/test_data/2024-01-25_22-19-10_test-f96b6e36/')

/var/folders/by/f4g6q8690z50zbfvy6kq71cc0000gn/T/ipykernel_80601/647576048.py:14: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if not nums:


In [26]:
gs = rec.streams['gaze']
ts = rec.streams['gaze'].ts
gs.sample([ts[0], ts[1], ts[2]])

for g in gs:
    print(g)

[(804.34216, 806.01044, 1.70621755e+09, 2.608845)]
[(802.19934, 807.4595, 1.70621755e+09, 2.61384869)]


In [27]:
rec.info
pprint.pprint(rec.info)

{'android_device_id': '6296df393d96a031',
 'android_device_model': 'rtwo_ge',
 'android_device_name': 'Neon Companion',
 'app_version': '2.7.10-prod',
 'calib_version': 1,
 'data_format_version': '2.2',
 'duration': 40252000000,
 'firmware_version': [29, 8],
 'frame_id': '1',
 'frame_name': 'Just act natural',
 'gaze_frequency': 200,
 'gaze_offset': [0.0, 0.0],
 'module_serial_number': '619453',
 'os_version': '13',
 'pipeline_version': '2.6.0',
 'recording_id': 'f96b6e36-6c98-409c-be7c-46c3e408acb2',
 'start_time': 1706217550686000000,
 'template_data': {'data': {'c700c50c-cee1-45fb-b459-050e5ceeaeae': ['']},
                   'id': '94733ede-ce4c-46be-aa72-7bb80e21bfe6',
                   'recording_name': '2024-01-25_22:19:10',
                   'version': '2024-01-25T09:33:35.681129Z'},
 'wearer_id': '65aee704-fa44-4fec-a811-6b4b126afdb0',
 'wearer_ied': 63.0,
 'workspace_id': 'e8984f4b-fe9f-48f5-bf3e-46e1ae0669b8'}


In [28]:
rec._calib

array([(1, b'619453', [[885.15825992,   0.        , 824.12887524], [  0.        , 885.14923875, 612.28065252], [  0.        ,   0.        ,   1.        ]], [-0.13053092,  0.10917948, -0.0001823 , -0.00043524,  0.00068462,  0.17075814,  0.05227603,  0.02661005], [[1., 0., 0., 0.], [0., 1., 0., 0.], [0., 0., 1., 0.], [0., 0., 0., 1.]], [[139.37418637,   0.        ,  99.23202584], [  0.        , 139.35188665,  96.19039145], [  0.        ,   0.        ,   1.        ]], [ 3.68394045e-02, -1.23725784e-01,  1.05156082e-03, -6.51159021e-04, -6.49958070e-01, -5.22825142e-02,  9.73858508e-03, -6.94081020e-01], [[-0.82693827,  0.18204613,  0.53200775, 16.86289978], [ 0.07135105,  0.9724648 , -0.22185867, 19.95415497], [-0.55774736, -0.14550412, -0.81715757, -7.03173828], [ 0.        ,  0.        ,  0.        ,  1.        ]], [[139.02147562,   0.        ,  93.94689674], [  0.        , 139.25351257,  96.6396657 ], [  0.        ,   0.        ,   1.        ]], [ 0.05170779, -0.11790007, -0.00186199, 

In [29]:
rec.streams

{'gaze': <__main__.Stream at 0x147d74210>,
 'imu': <__main__.Stream at 0x147d74550>,
 'scene': <__main__.Stream at 0x147d75110>}

In [30]:
rec.streams['gaze'].data

rec.array([(806.6711 ,  806.5423 , 1.70621755e+09,  2.60384369),
           (804.34216,  806.01044, 1.70621755e+09,  2.608845  ),
           (802.19934,  807.4595 , 1.70621755e+09,  2.61384869), ...,
           (883.96313, 1085.5363 , 1.70621759e+09, 40.2386837 ),
           (887.2845 , 1084.7004 , 1.70621759e+09, 40.24368477),
           (884.99097, 1087.0084 , 1.70621759e+09, 40.24882483)],
          dtype=[('x', '<f4'), ('y', '<f4'), ('ts', '<f8'), ('ts_rel', '<f8')])

In [31]:
rec.start_ts

1706217550.6860003

In [32]:
rec.streams['gaze'].data.ts[0] - rec.start_ts

2.6038436889648438

In [33]:
rec.streams['gaze'].data.ts_rel

array([ 2.60384369,  2.608845  ,  2.61384869, ..., 40.2386837 ,
       40.24368477, 40.24882483])

In [34]:
rec.streams['imu'].data.ts

array([1.70621755e+09, 1.70621755e+09, 1.70621755e+09, ...,
       1.70621759e+09, 1.70621759e+09, 1.70621759e+09])

In [35]:
[rec.streams['imu'].data.ts[0], rec.streams['imu'].data.ts[1]]

[1706217553.1487713, 1706217553.1590152]

In [36]:
for stream in rec.streams:
    s = rec.streams[stream]
    name = s.name
    print(name)
    avg_spf = (s.data.ts[-1] - s.data.ts[0]) / len(s.data.ts)
    avg_fps = 1 / avg_spf
    print(avg_fps)
    start_offset = s.data.ts[0] - rec.start_ts
    print(start_offset)
    print()

gaze
198.69846583315015
2.6038436889648438

imu
104.15957776951906
2.462770938873291

scene


AttributeError: 'list' object has no attribute 'ts'

In [37]:
np.all((rec.gaze.data == rec.streams['gaze'].data).flatten())

rec.array(True,
          dtype=bool)

In [38]:
np.all((rec.imu.data == rec.streams['imu'].data).flatten())

rec.array(True,
          dtype=bool)

In [39]:
rec.gaze.ts

array([1.70621755e+09, 1.70621755e+09, 1.70621755e+09, ...,
       1.70621759e+09, 1.70621759e+09, 1.70621759e+09])

In [40]:
np.all((rec.gaze.ts == rec.streams['gaze'].ts).flatten())

True

In [41]:
np.all((rec.imu.ts == rec.streams['imu'].ts).flatten())

True

In [42]:
np.all(rec.imu.ts_rel == rec.streams['imu'].ts_rel)

True

In [43]:
np.all(rec.gaze.ts_rel == rec.streams['gaze'].ts_rel)

True

In [44]:
# adapted from here:
# https://github.com/pupil-labs/neon-player/blob/master/pupil_src/shared_modules/pupil_recording/update/neon.py#L181

rec_dir = '../tests/test_data/2024-01-25_22-19-10_test-f96b6e36/'
rec_path = pathlib.Path(rec_dir)

labels = (rec_path / 'event.txt').read_text().strip().split('\n')

_rewrite_times(pathlib.Path(rec_dir + '/event.time'))
timestamps = np.load(rec_path / 'event_timestamps.npy')
timestamps_unix = np.load(rec_path / 'event_timestamps_unix.npy')

In [45]:
labels

['recording.begin', 'recording.end']

In [46]:
timestamps

array([1.70621755e+09, 1.70621759e+09])

In [47]:
timestamps_unix

array([1706217550686000000, 1706217590938000000], dtype=uint64)

In [48]:
events = [evt for evt in zip(labels, timestamps)]
events

[('recording.begin', 1706217550.6860003), ('recording.end', 1706217590.938)]

In [49]:
unique_events = dict(events)
unique_events

{'recording.begin': 1706217550.6860003, 'recording.end': 1706217590.938}

In [50]:
unique_events['recording.begin']

1706217550.6860003

In [51]:
t = [
    ('rec.b', 1706217550.6860003),
    ('rec.b', 1706217888.6860003),
    ('rec.c', 1706218888.938),
    ('rec.c', 1706217590.938),
]
t

[('rec.b', 1706217550.6860003),
 ('rec.b', 1706217888.6860003),
 ('rec.c', 1706218888.938),
 ('rec.c', 1706217590.938)]

In [52]:
t.reverse()
t

[('rec.c', 1706217590.938),
 ('rec.c', 1706218888.938),
 ('rec.b', 1706217888.6860003),
 ('rec.b', 1706217550.6860003)]

In [53]:
dict(t)

{'rec.c': 1706218888.938, 'rec.b': 1706217550.6860003}

In [1]:
import pupil_labs.neon_recording as nr

rec = nr.load('../tests/test_data/2024-01-25_22-19-10_test-f96b6e36/')

gaze_data = rec.streams['gaze'].data
gaze_ts = rec.streams['gaze'].ts
gaze_start_offset = rec.streams['gaze'].data.ts[0] - rec.start_ts
# there is also rec.streams['gaze'].ts_rel

gaze = rec.streams['gaze']
gaze.sample([gaze_ts[0], gaze_ts[1], gaze_ts[2]])

for g in gaze:
    if g:
        print(g.x, g.y, g.ts)

print()
sample_ts = gaze_ts[:15]
for gaze, imu in zip(rec.gaze.sample(sample_ts), rec.imu.sample(sample_ts)):
    if gaze:
        x = gaze.x
        y = gaze.y
        ts = gaze.ts

        print('gz', x, y, ts)


    if imu:
        pitch = imu.pitch
        yaw = imu.yaw
        roll = imu.roll
        ts = imu.ts

        print('imu', pitch, yaw, roll, ts)


NeonRecording: Loading recording from: ../tests/test_data/2024-01-25_22-19-10_test-f96b6e36
NeonRecording: Loading recording info and calibration data
NeonRecording: Loading raw time files
NeonRecording: Loading 'gaze ps1' data
NeonRecording: Loading 'gaze_200hz' data
NeonRecording: Loading 'gaze_right_ps1' data
NeonRecording: Loading IMU data
NeonRecording: Loading events
NeonRecording: Parsing unique events
NeonRecording: Finished loading recording.

[804.34216309] [806.01043701] [1.70621755e+09]
[802.19934082] [807.45947266] [1.70621755e+09]

gz [804.34216309] [806.01043701] [1.70621755e+09]
gz [802.19934082] [807.45947266] [1.70621755e+09]
imu [-118.25288] [-44.7666] [124.22863] [1.70621755e+09]
gz [800.52740479] [803.5536499] [1.70621755e+09]
gz [804.71148682] [802.50836182] [1.70621755e+09]
imu [-118.21066] [-44.766228] [124.16915] [1.70621755e+09]
gz [804.94714355] [802.89910889] [1.70621755e+09]
gz [803.04333496] [808.11279297] [1.70621755e+09]
gz [805.105896] [805.82208252] [1

In [2]:
rec.gaze.sample([gaze_ts[0], gaze_ts[1], gaze_ts[2]])
for g in rec.gaze.try_generator():
    print(g)

AttributeError: 'Stream' object has no attribute 'try_generator'

In [3]:
rec.imu.sample_one(gaze_ts[0])

array((-0.81443787, -1.1463165, -0.45204163, 0.04101563, -0.6098633, 0.7675781, -118.2757, -44.769444, 124.262, 0.51070815, -0.19833069, 0.26649785, -0.79298234, 1706217553288873064, 1.70621755e+09, 2.60287261),
      dtype=(numpy.record, [('gyro_x', '<f4'), ('gyro_y', '<f4'), ('gyro_z', '<f4'), ('accel_x', '<f4'), ('accel_y', '<f4'), ('accel_z', '<f4'), ('pitch', '<f4'), ('yaw', '<f4'), ('roll', '<f4'), ('quaternion_w', '<f4'), ('quaternion_x', '<f4'), ('quaternion_y', '<f4'), ('quaternion_z', '<f4'), ('tsNs', '<u8'), ('ts', '<f8'), ('ts_rel', '<f8')]))

In [5]:
rec.gaze.to_numpy()

array([[8.04342163e+02, 8.06010437e+02, 1.70621755e+09, 2.60884500e+00],
       [8.02199341e+02, 8.07459473e+02, 1.70621755e+09, 2.61384869e+00]])